In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
# Dec 15 games

# NOTE: Odds boosted to +105 for a patriots, chiefs, sf49ers outcome
# line 10.5
bengals = MoneyLine(event="bengals", bet_amount=100, odds=420)
patriots = MoneyLine(event="patriots", bet_amount=100, odds=-500)

# line 9.5
broncos = MoneyLine(event="broncos", bet_amount=100, odds=370)
chiefs = MoneyLine(event="chiefs", bet_amount=100, odds=-435)

# line 10
falcons = MoneyLine(event="falcons", bet_amount=100, odds=360)
sf49ers = MoneyLine(event="sf49ers", bet_amount=100, odds=-435)

binaries = [
    [bengals, patriots],
    [broncos, chiefs],
    [falcons, sf49ers],
]

results = [
    [0, 1], #[patriots, bengals],
    [0, 1], #[broncos, chiefs],
    [1, 0], #[falcons, sf49ers],
]

override_arr = [1, 3, 5]


def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(binaries, results)

In [3]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(binaries)
SELECT_NUM = 3
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [4]:
len(all_subsets)

1

In [5]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=0.50,
                    bounds=(0.01, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup,
                    override_arr=override_arr
                    )
    
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

yooooo patriots_chiefs_sf49ers [1, 3, 5] 105 2.05
     fun: -0.39580164929568085
     jac: array([-1.31780000e+01, -2.88229885e+00, -2.80235632e+00, -1.08201213e-01,
       -2.36800000e+00,  7.93103501e-03,  2.63793096e-02,  6.18750002e-01])
 message: 'Positive directional derivative for linesearch'
    nfev: 210
     nit: 16
    njev: 12
  status: 8
 success: False
       x: array([0.01      , 0.03088065, 0.03155197, 0.10665968, 0.03577793,
       0.10209613, 0.10125622, 0.01      ])
slsqp_solver: 
                      event    index[]     result  event_status   bet        mult  payout  profit
6   patriots_chiefs_falcons  [1, 3, 4]  [1, 1, 1]          True  0.10    6.788966  0.6874    0.26
0   bengals_broncos_falcons  [0, 2, 4]  [0, 0, 1]         False  0.01  112.424000  1.1242    0.70
1   bengals_broncos_sf49ers  [0, 2, 5]  [0, 0, 0]         False  0.03   30.058391  0.9282    0.50
2    bengals_chiefs_falcons  [0, 3, 4]  [0, 1, 1]         False  0.03   29.418851  0.9282    0.50
4  pa

In [6]:
df.describe()

,bet,mult,payout,profit
count,8.000000,8.000000,8.000000,8.000000
mean,0.053750,27.685796,0.770475,0.342500
std,0.042405,36.074273,0.333312,0.334611
min,0.010000,2.050000,0.020500,-0.410000
25%,0.025000,6.899655,0.703000,0.275000
50%,0.035000,16.904805,0.883550,0.455000
75%,0.100000,29.578736,0.928200,0.500000
max,0.110000,112.424000,1.124200,0.700000


In [7]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['event_status', 'profit'], ascending=[False, True])
negative_df.describe()

,bet,mult,payout,profit
count,1.00,1.00,1.0000,1.00
mean,0.01,2.05,0.0205,-0.41
std,NaN,NaN,NaN,NaN
min,0.01,2.05,0.0205,-0.41
25%,0.01,2.05,0.0205,-0.41
50%,0.01,2.05,0.0205,-0.41
75%,0.01,2.05,0.0205,-0.41
max,0.01,2.05,0.0205,-0.41


In [8]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['event_status', 'profit'], ascending=[False, False])
positive_df.describe()

,bet,mult,payout,profit
count,7.000000,7.000000,7.000000,7.000000
mean,0.060000,31.348053,0.877614,0.450000
std,0.041633,37.323793,0.149924,0.150886
min,0.010000,6.788966,0.687400,0.260000
25%,0.030000,7.401081,0.773550,0.345000
50%,0.040000,25.944000,0.928200,0.500000
75%,0.100000,29.738621,0.928200,0.500000
max,0.110000,112.424000,1.124200,0.700000


In [9]:
negative_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
7,patriots_chiefs_sf49ers,"[1, 3, 5]","[1, 1, 0]",False,0.01,2.05,0.0205,-0.41


In [10]:
positive_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
6,patriots_chiefs_falcons,"[1, 3, 4]","[1, 1, 1]",True,0.10,6.788966,0.6874,0.26
0,bengals_broncos_falcons,"[0, 2, 4]","[0, 0, 1]",False,0.01,112.424000,1.1242,0.70
1,bengals_broncos_sf49ers,"[0, 2, 5]","[0, 0, 0]",False,0.03,30.058391,0.9282,0.50
2,bengals_chiefs_falcons,"[0, 3, 4]","[0, 1, 1]",False,0.03,29.418851,0.9282,0.50
4,patriots_broncos_falcons,"[1, 2, 4]","[1, 0, 1]",False,0.04,25.944000,0.9282,0.50
3,bengals_chiefs_sf49ers,"[0, 3, 5]","[0, 1, 0]",False,0.11,7.865610,0.8389,0.41
5,patriots_broncos_sf49ers,"[1, 2, 5]","[1, 0, 0]",False,0.10,6.936552,0.7082,0.28


In [11]:
positive_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,6,6,6,6,6,6,6
True,1,1,1,1,1,1,1


In [12]:
negative_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,1,1,1,1,1,1,1


In [13]:
neg = negative_df.reset_index(drop=True).style.applymap(color_negative_red)
neg

,event,index[],result,event_status,bet,mult,payout,profit
0,patriots_chiefs_sf49ers,"['1', '3', '5']","[1, 1, 0]",False,0.01,2.05,0.0205,-0.41


In [14]:
pos = positive_df.reset_index(drop=True).style.applymap(color_positive_green)
pos

,event,index[],result,event_status,bet,mult,payout,profit
0,patriots_chiefs_falcons,"['1', '3', '4']","[1, 1, 1]",True,0.1,6.78897,0.6874,0.26
1,bengals_broncos_falcons,"['0', '2', '4']","[0, 0, 1]",False,0.01,112.424,1.1242,0.7
2,bengals_broncos_sf49ers,"['0', '2', '5']","[0, 0, 0]",False,0.03,30.0584,0.9282,0.5
3,bengals_chiefs_falcons,"['0', '3', '4']","[0, 1, 1]",False,0.03,29.4189,0.9282,0.5
4,patriots_broncos_falcons,"['1', '2', '4']","[1, 0, 1]",False,0.04,25.944,0.9282,0.5
5,bengals_chiefs_sf49ers,"['0', '3', '5']","[0, 1, 0]",False,0.11,7.86561,0.8389,0.41
6,patriots_broncos_sf49ers,"['1', '2', '5']","[1, 0, 0]",False,0.1,6.93655,0.7082,0.28


In [15]:
sf49ers = MoneyLine(event="parlay", bet_amount=0.01, odds=110)
sf49ers.print_stats()

    event  bet_amount  odds  multiplier  payout
0  parlay        0.01   110         2.1   0.021
